In [103]:
import sys
sys.path.append('../')
import config
from collections import OrderedDict, defaultdict
from models import *
from models.CharTransE import *
import numpy as np
from scipy.sparse.csgraph import connected_components
from scipy.sparse import csr_matrix

In [86]:
con = config.Config()
con.set_in_path("../benchmarks/tim_minikb/")
con.set_result_dir("../result/")
con.init()

In [129]:
ent2id = OrderedDict()
id2ent = OrderedDict()
with open( con.in_path + 'entity2id.txt') as nf:
    lines = [line.split('\t') for line in nf.readlines()[1:]]
    ent2id = {line[0]:int(line[1]) for line in lines}
    id2ent = {int(line[1]):line[0] for line in lines}
trip_dict_names = defaultdict(list) 
with open(con.in_path + 'rel_triples.txt') as rf:
    triples = [line.split('\t') for line in rf.readlines()]
    for trip in triples:
        trip_dict_names[trip[0]] += [trip]
        
trip_dict = defaultdict(list) 
with open(con.in_path + 'train2id.txt') as rf:
    triples = [[int(el) for el in line.split(' ')] for line in rf.readlines()[1:]]
    for trip in triples:
        trip_dict[trip[0]] += [trip]
name_ids = [id_ for ent, id_ in ent2id.items() if not ent.startswith('http')]
ent_ids = [id_ for ent, id_ in ent2id.items() if 'entities' in ent]

In [88]:
cte = CharTransE(con)
cte.load_state_dict(torch.load(con.result_dir +  'CharTransE.ckpt'))

/media/rehan/yaws/ws_new/OpenKE/py/lib/python3.7/site-packages/torch/nn/_reduction.py:46: UserWarning: size_average and reduce args will be deprecated, please use reduction='sum' instead.
  warnings.warn(warning.format(ret))


IncompatibleKeys(missing_keys=[], unexpected_keys=[])

In [89]:
tfidf_tensor = sparse_tensor(cte.tf_idf_mat)
ent_emb_mat = cte.ent_char_weights(tfidf_tensor).cpu().data.numpy()
rel_emb_mat = cte.rel_embeddings.weight.data.numpy()

In [90]:
print(rel_emb_mat.shape)
print(ent_emb_mat.shape)

(38, 100)
(5741, 100)


In [91]:
name_ent_emb = ent_emb_mat[name_ids,:]
name_ent_emb = name_ent_emb/(np.linalg.norm(name_ent_emb, axis=1).reshape((-1,1)))

In [92]:
sim_mat = name_ent_emb.dot(name_ent_emb.T)

In [93]:
adj_mat = sim_mat > 0.8

In [94]:
n_components, labels = connected_components(csgraph=adj_mat, directed=False, return_labels=True)

In [95]:
clusters = defaultdict(list)
for lab, id_ in zip(labels, name_ids):
    clusters[lab] += [id_]

In [96]:
big_clusters = [val for val in clusters.values() if len(val) >= 2]

In [97]:
cluster_names = [[id2ent[i] for i in clus] for clus in big_clusters]

In [231]:
ent_embeddings = []
for ent_id in ent_ids:
    trips = trip_dict[ent_id]
    emb_rel = np.zeros(ent_emb_mat.shape[1])
    emb = ent_emb_mat[ent_id, :]
    name_emb = np.zeros(ent_emb_mat.shape[1])
    for t in trips:
#         head = t[0]
        tail = t[1]
        rel = t[2]
        tail_emb = ent_emb_mat[tail, :]
        rel_emb = rel_emb_mat[rel, :]
        if rel == 1:
#             emb_rel = emb_rel + tail_emb
            name_emb = tail_emb
        else:
            emb_rel = emb_rel + (emb*tail_emb*rel_emb)
#         emb_rel = emb_rel + (ent_emb_mat[tail, :] - rel_emb_mat[rel, :])
        
#     emb = emb + (emb_rel/len(trips))
    ent_embeddings.append(emb + name_emb)
ent_embeddings = np.array(ent_embeddings)
ent_embeddings = ent_embeddings/(np.linalg.norm(ent_embeddings, axis=1).reshape((-1,1)))
ent_sim_mat = ent_embeddings.dot(ent_embeddings.T)
ent_adj_mat = csr_matrix(ent_sim_mat > 0.95)

In [232]:
np.sum(ent_sim_mat - np.eye(ent_sim_mat.shape[0]) > 0.95)

8124

In [233]:
def get_connected_components(adjacency_mat):
    n_components, labels = connected_components(csgraph=adjacency_mat, directed=False, return_labels=True)
    clusters = defaultdict(list)
    for id_, lab in enumerate(labels):
        clusters[lab] += [id_]
    return clusters.values()

In [234]:
ent_clusters = get_connected_components(ent_adj_mat)

In [235]:
ent_clusters_big = [clus for clus in ent_clusters if len(clus) >= 2]

In [236]:
ent_clusters_big

[[2,
  7,
  62,
  298,
  414,
  531,
  818,
  1199,
  1543,
  1597,
  2038,
  2072,
  2331,
  2395,
  2490],
 [10, 312, 2425],
 [14,
  21,
  23,
  24,
  32,
  34,
  37,
  42,
  46,
  51,
  70,
  76,
  78,
  83,
  95,
  101,
  119,
  122,
  126,
  132,
  143,
  145,
  158,
  161,
  171,
  176,
  179,
  189,
  193,
  201,
  220,
  223,
  250,
  255,
  261,
  266,
  267,
  271,
  280,
  285,
  299,
  303,
  308,
  311,
  315,
  320,
  352,
  359,
  364,
  370,
  371,
  382,
  384,
  386,
  404,
  405,
  412,
  429,
  437,
  438,
  440,
  450,
  456,
  461,
  480,
  500,
  503,
  507,
  513,
  514,
  516,
  526,
  529,
  532,
  542,
  549,
  558,
  563,
  577,
  578,
  579,
  581,
  601,
  609,
  612,
  617,
  620,
  626,
  628,
  632,
  643,
  646,
  652,
  669,
  680,
  681,
  692,
  693,
  698,
  699,
  700,
  703,
  709,
  713,
  720,
  727,
  735,
  743,
  761,
  767,
  772,
  782,
  788,
  791,
  792,
  801,
  802,
  806,
  820,
  821,
  823,
  830,
  831,
  836,
  841,
  843,
  858,

In [250]:
trip_dict[ent_ids[265]]
trip_dict_names[id2ent[ent_ids[391]]]

[['http://www.isi.edu/gaia/entities/2473ce1d-14cb-43a9-aa62-7c5c34275f47',
  'http://www.w3.org/1999/02/22-rdf-syntax-ns#type',
  'https://tac.nist.gov/tracks/SM-KBP/2018/ontologies/SeedlingOntology#Facility\n'],
 ['http://www.isi.edu/gaia/entities/2473ce1d-14cb-43a9-aa62-7c5c34275f47',
  'https://tac.nist.gov/tracks/SM-KBP/2018/ontologies/SeedlingOntology#Physical.LocatedNear_Place',
  'http://www.isi.edu/gaia/entities/58b98e73-a202-4047-b70d-e19d59805dfd\n'],
 ['http://www.isi.edu/gaia/entities/2473ce1d-14cb-43a9-aa62-7c5c34275f47',
  'https://tac.nist.gov/tracks/SM-KBP/2018/ontologies/SeedlingOntology#Physical.LocatedNear_Place',
  'http://www.isi.edu/gaia/entities/77e412d7-2488-4481-b118-9201bec52b0d\n'],
 ['http://www.isi.edu/gaia/entities/2473ce1d-14cb-43a9-aa62-7c5c34275f47',
  'https://tac.nist.gov/tracks/SM-KBP/2018/ontologies/InterchangeOntology#hasName',
  'prison\n']]

In [251]:
trip_dict[ent_ids[1540]]
trip_dict_names[id2ent[ent_ids[1702]]]

[['http://www.isi.edu/gaia/entities/a31efe29-5830-4202-a7ca-52a37ec4d972',
  'https://tac.nist.gov/tracks/SM-KBP/2018/ontologies/SeedlingOntology#Physical.LocatedNear_Place',
  'http://www.isi.edu/gaia/entities/b561952a-ad79-49a1-a62b-3640a44349b0\n'],
 ['http://www.isi.edu/gaia/entities/a31efe29-5830-4202-a7ca-52a37ec4d972',
  'https://tac.nist.gov/tracks/SM-KBP/2018/ontologies/InterchangeOntology#hasName',
  'prison\n'],
 ['http://www.isi.edu/gaia/entities/a31efe29-5830-4202-a7ca-52a37ec4d972',
  'http://www.w3.org/1999/02/22-rdf-syntax-ns#type',
  'https://tac.nist.gov/tracks/SM-KBP/2018/ontologies/SeedlingOntology#Facility\n']]